In [1]:
import requests
from bs4 import BeautifulSoup
import time 
import random 
import re
from fake_useragent import UserAgent
import datetime
from elasticsearch import Elasticsearch
from tqdm import tqdm
es=Elasticsearch()

In [2]:
class KanobuArticle:
    def __init__(self):
        self.date=""
        self.time=""
        self.words=""
        self.rubr=""
        self.hashtag=""
        self.author=""
        self.head=""
        self.text=""
        

    def toJSON(self):
        res='{"date":"'+self.date+'", "time":"'+self.time+'", "words":"'+self.words+'", "rubrics":"'+self.rubr+'", "hashtag":"'+self.hashtag+'", "title":"'
        +self.head+'", "author":"'+self.author+'","text":"'
        res+=self.text.replace('"', '\\"')+'"}'
        return res


    def toDict(self):
        res={"date":self.date, "time":self.time, "rubrics":self.rubr, "hashtag":self.hashtag, \
             "title":self.head, "words":self.words, "author":self.author,"text":self.text.replace('"', '\\"')}
        return res

In [3]:
def GetPagesArticles(number_of_pages):
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.4; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2225.0 Safari/537.36'}
    url = 'https://kanobu.ru/news/?page='
    links = []
    for page in range(1, number_of_pages+1):
        r = requests.get(url + str(page), headers = headers)
        time.sleep(random.randint(1,6))
        content = r.content
        html = content.decode('UTF-8')
        soup = BeautifulSoup(html)
        
        for link in soup.find_all('a', attrs={'class':'c-item_foot d-b t-l'}):
            links.append(link.get('href'))
    
    return links    

In [4]:
def clean_text (text):
    text = text.replace('\xa0', ' ')
    text = text.replace ('\n', ' ')
    text = text.replace ('  ', ' ')
    return text

In [302]:
def add_article(link):
    art = KanobuArticle()
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.4; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2225.0 Safari/537.36'}
    page = requests.get(link, headers = headers)
    content = page.content
    html = content.decode('UTF-8')
    soup = BeautifulSoup(html)
    
    date = soup.find('div', attrs={'class':'c-title_footer'}).get_text()
    art.date = date.split(', ')[0]
    art.time = date.split(', ')[1]
        
    author = soup.find('a', attrs={'class':'c-title_author'}).get_text()
    art.author = author
        
    text = soup.find('div', attrs={'class':'c-detail_content'}).get_text()
    art.text = clean_text(text)
        
    words = len(clean_text(text).split())
    art.words = words
        
    title = soup.find('h1', attrs={'class':'c-title_body'}).get_text()
    art.head = clean_text(title)
        
    rubric = soup.find('div', attrs={'class':'c-title_head t-t-u'}).get_text()
    r = rubric.split(' | ')
    art.rubr = r[0]
    try:
        art.hashtag = r[1]
    except:
        pass
    
    #print(art)
    #print(text)
    #print(date)
    #print(author)
    print(title)
    print(words)
    #print(r[0])
    #print(r[1])
    
    
    res = es.search(index="kanobu", body={"query": {"match_all": {}}})
    maxid = res['hits']['total']['value']
    
    es.index(index="kanobu", doc_type='article', id = maxid, body=art.toDict())

In [241]:
def create_db(index):
    es.indices.create(index=index)
    mapit={"article":{"properties":{"author":{"type":"text"},
                                "date":{"type":"text"},
                                "time":{"type":"date", "format":" HH:mm"},
                                "words":{"type":"double"},
                                "hashtag":{"type":"text","analyzer" : "russian"},
                                "rubrics":{"type":"text","analyzer" : "russian"},
                                "text":{"type":"text","analyzer" : "russian"},
                                "title":{"type":"text","analyzer" : "russian"}}}}
    es.indices.put_mapping(index=index, doc_type='article', body=mapit, include_type_name=True)

In [303]:
def GetArticles(links):
    create_db("kanobu")
    l = 'https://kanobu.ru'
    for link in links:
        add_article(l+link)
        time.sleep(random.randint(1,3))

In [304]:
es.indices.delete(index="kanobu") 

{'acknowledged': True}

In [305]:
GetArticles(GetPagesArticles(2))

Владельцам Call of Duty: Warzone бесплатно доступен мультиплеер Modern Warfare
192
Представлены смарт-камеры серии Xiaomi Mi Smart Camera PTZ: новинки для создания безопасного дома
140
Дэниел Крейг, Кейт Уинслет, Элтон Джон и другие знаменитости поблагодарили медиков
140
Marvel открыла бесплатный доступ к части своих комиксов. Но только до 4 мая
156
«Он тупой»: Хабиб ответил на требование Фергюсона лишить его чемпионского пояса
201
Okko открыл доступ к бесплатной коллекции фильмов и концертов
179
Российский дизайнер сделал глянцевый журнал о коронавирусе с пациентами больницы
199
Николь Кидман сыграет в новом триллере от Amazon
164
Инсайдер объяснил, почему перенесли The Last of Us 2
173
Xiaomi представила новые 4К-телевизоры на 60 и 75 дюймов
175
Танцы с гробом стали новым мемом. Даже пародию в Doom Eternal сделали
201
Джеймс Ганн раскрыл сюжет «Скуби-Ду 3». Фанаты приятно удивлены
178
Xiaomi представила дорогие детские смарт-часы Mi Bunny Childrenʼs Watch 4
169
Фредди Крюгер советует

### Интерфейс

In [276]:
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output
from ipywidgets import interact, interactive, fixed, interact_manual

In [277]:
text1 = widgets.Text(
    value='',
    placeholder='Type what you want to find',
    description='Text:',
    disabled=False
)

In [278]:
menu = widgets.Dropdown(
       options=['id', 'title', 'date', 'author', 'hashtag', 'rubrics'],
       value='id',
       description='How to find?')

In [279]:
text = widgets.Text(
    value='',
    placeholder='Type what you want to find',
    description='Text:',
    disabled=False
)

In [280]:
text2 = widgets.Text(
    value='',
    placeholder='Enter the link of the article to add',
    description='Text:',
    disabled=False
)

In [281]:
order = widgets.Dropdown(
       options=['ascending', 'descending'],
       value='ascending',
       description='Order')

In [282]:
sortby = widgets.Dropdown(
       options=['_id', 'words'],
       value='_id',
       description='Sort by')

In [283]:
number = widgets.IntSlider(
    value=10,
    min=1,
    max=10000,
    step=1,
    description='How many articles?',
)



In [284]:
counter = widgets.Dropdown(
       options=['avg', 'max'],
       value='max',
       description='words')

In [285]:
button1 = widgets.Button(description='Count')
out = widgets.Output()

In [286]:
def on_button_clicked1(_):
    with out:
        clear_output()
        res=es.search(index = "kanobu", body = {"size": 0, "aggs": {counter.value: {counter.value: {"field": "words" }}}})
        print(round(res["aggregations"][counter.value]["value"]))
        

In [287]:
button1.on_click(on_button_clicked1)

In [288]:
button2 = widgets.Button(description='Match with options')
out = widgets.Output()

def on_button_clicked2(_):
    with out:
        clear_output()
        if menu.value == 'id':
            id_search = int(text.value)
            print(es.get(index="kanobu", doc_type='article', id=id_search))
        
        else:
            res = es.search(index="kanobu", body={"query": {"match": {menu.value : text.value}}}, size=100)
            for part in res["hits"]["hits"]:
                print(part)

In [289]:
button2.on_click(on_button_clicked2)

In [290]:
button3 = widgets.Button(description='Add article')
out = widgets.Output()

def on_button_clicked3(_):
    with out:
        clear_output()
        link = text2.value
        add_article(link)

In [291]:
button3.on_click(on_button_clicked3)

In [292]:
text3 = widgets.Text(
    value='',
    placeholder='Enter article id to delete',
    description='Text:',
    disabled=False
)

In [293]:
button4 = widgets.Button(description='Delete article')
out = widgets.Output()

def on_button_clicked4(_):
    with out:
        clear_output()
        id_del = int(text3.value)
        es.delete(index="kanobu",doc_type="article",id=id_del)
        print('Done')

In [294]:
button4.on_click(on_button_clicked4)

In [295]:
button5 = widgets.Button(description='Sort articles')
out = widgets.Output()

In [296]:
def on_button_clicked5(_):
    with out:
        clear_output()
        if order.value == 'ascending':
            ord = str(sortby.value)+':asc'
        elif order.value == 'descending':
            ord = str(sortby.value)+':desc'
        res = es.search(index="kanobu", body={"query": {"match_all": {}}}, sort=ord,size=number.value)
        for part in res["hits"]["hits"]:
            print(part)

In [297]:
button5.on_click(on_button_clicked5)

In [298]:
id_upd = widgets.Text(
    value='',
    placeholder='Enter your id',
    description='Text:',
    disabled=False
)

menu_upd = widgets.Dropdown(
       options=['title', 'date', 'author', 'hashtag', 'rubrics', 'text'],
       value='title',
       description='What to update?')

text_upd = widgets.Text(
    value='',
    placeholder='Enter your update',
    description='Text:',
    disabled=False
)

button_upd = widgets.Button(description='Update')
out = widgets.Output()

def on_button_clicked_upd(_):
    with out:
        clear_output()
        your_id = int(id_upd.value)
        upd_area = str(menu_upd.value)
        text = str(text_upd.value)
        es.update(index='kanobu', doc_type='article', id=your_id, body={"doc": {upd_area: text}})
        print('Done')

In [299]:
button_upd.on_click(on_button_clicked_upd)

In [306]:
box1 = widgets.VBox([menu, text, button2])
box2 = widgets.VBox([text2, button3, text3, button4])
box3 = widgets.VBox([id_upd, menu_upd, text_upd, button_upd])
box4 = widgets.VBox([sortby,order, number, button5])
box5 = widgets.VBox([counter, button1])
widgets.VBox([box1, box2, box3, box4, box5, out])

- поиск по Id выполняется по другому (придумать как добавить его в ту же кнопку) - <b>сделано</b>
- кнопка для добавления статьи по ссылке - <b>сделано</b>
- не хватает операции update (например, "положи по существующему индексу другую статью" - <b>сделано</b>
- нужно организовать меню с кнопками получше - <b>сделано</b>
- сортировка <b>сделано</b>
- - "2 - два действия помимо CRUD (сортировка, группировка, агрегация, ...) " <b>сделано</b>
- виджет с агрегацией <b>сделано</b>
- Когда ищу для 2 апреля сначала идут статьи 2 апреля, а потом почему-то статьи 3 апреля (и наоборот)
- группировка 

"Требования к нереляционным БД
2 - нереляционная БД
2 - красивая структура БД
2 - интерфейс позволяет класть, доставать, удалять данные (проводить операции CRUD - Create, Read, Update, Delete)
2 - два действия помимо CRUD (сортировка, группировка, агрегация, ...)
2 - зависит от БД: Redis - использование ключей, хешей, ...; Neo4j - нахождение путей не только к соседним вершинам, операции с графами; MongoDB, ElasticSearch - работа с текстами или географией; остальные БД - будем договариваться."